In [23]:
!pip install transformers torch sentencepiece


In [24]:
from transformers import pipeline
llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=128
)




Device set to use cpu


In [25]:
doctor_schedule = {
    "2026-01-02": ["10:00", "11:00", "15:00"],
    "2026-01-03": ["09:00", "13:00"],
}


In [26]:
def availability_tool(date):
    slots = doctor_schedule.get(date, [])
    if slots:
        return f"Available slots on {date}: {', '.join(slots)}"
    return "No slots available on this date."


In [27]:
def booking_tool(date, time):
    if date in doctor_schedule and time in doctor_schedule[date]:
        doctor_schedule[date].remove(time)
        return f"---Appointment confirmed on {date} at {time}.--- \n Address: #1, 2nd Main, Yadavnagar, Bangalore."
    return "!! Selected slot is not available. !!"


In [28]:
def chat_tool(user_input):
    prompt = f"""
You are a polite hospital assistant.
Your job is to help users with doctor appointments.
Nearest medical store is Murugan Medicals, 2nd main road.
Clinic timings are between 9am and 5pm.

Rules:
- Do not repeat user input.
- Do not invent appointment slots.
- Be short, polite, and helpful.
- Answer user queries.

User: {user_input}
Assistant:
"""
    return llm(prompt)[0]["generated_text"].strip()


In [29]:
import re
def decide_and_act(user_input):
    prompt = f"""
You are an AI agent with tools:
1. CHECK_AVAILABILITY
2. BOOK_APPOINTMENT
3. GENERAL_CHAT

Rules:
- If user asks about available slots, respond exactly: CHECK_AVAILABILITY
- If user wants to book an appointment, respond exactly: BOOK_APPOINTMENT
- Otherwise, respond exactly: GENERAL_CHAT

Do not explain anything. Only output one of these: CHECK_AVAILABILITY, BOOK_APPOINTMENT, GENERAL_CHAT

User Input: {user_input}
Decision:
"""
    decision = llm(prompt)[0]["generated_text"].strip()

    # Extract date and time
    date_match = re.search(r"\d{4}-\d{2}-\d{2}", user_input)
    time_match = re.search(r"\d{2}:\d{2}", user_input)

    if decision == "CHECK_AVAILABILITY" and date_match:
        return availability_tool(date_match.group())

    elif decision == "BOOK_APPOINTMENT" and date_match and time_match:
        return booking_tool(date_match.group(), time_match.group())

    else:
        return chat_tool(user_input)


In [30]:
print("Doctor Appointment AI Agent for Dr. Amar  \
Clinic(Dermatologist) (type 'exit' to quit)")
while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Agent: Thank you! Stay healthy 😊")
        break

    response = decide_and_act(user_input)
    print("Agent:", response)


Doctor Appointment AI Agent for Dr. Amar  Clinic(Dermatologist) (type 'exit' to quit)
You: Hello
Agent: Hello. How can I help you?
You: Are there any available slots on 2026-01-02?
Agent: Available slots on 2026-01-02: 10:00, 11:00, 15:00
You: I would like to book an appointment on 2026-01-02 at 11:00
Agent: ---Appointment confirmed on 2026-01-02 at 11:00.--- 
 Address: #1, 2nd Main, Yadavnagar, Bangalore.
You: I want to book an appointment on 2026-01-02 at 11:00 again
Agent: !! Selected slot is not available. !!
You: Please let me know available slots on 2026-01-06
Agent: No slots available on this date.
You: What are the clinic timings?
Agent: Answer: Clinic timings are between 9am and 5pm.
You: Where is the nearest medical store?
Agent: Answer: Murugan Medicals, 2nd main road.
You: exit
Agent: Thank you! Stay healthy 😊
